In [11]:
import os
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#import pyarrow as pa
#import datetime

# Define paths
wea_files = ['LC_2022Q1.csv', 'LC_2022Q2.csv', 'LC_2022Q3.csv', 'LC_2022Q4.csv']
mp_list = ['MP 01', 'MP 02', 'MP 03', 'MP 04', 'MP 05', 'MP 06', 'MP 07', 'MP 08']
mp_list2 = ['MP 01', 'MP 02', 'MP 03', 'MP 05', 'MP 06', 'MP 07']
id_list = ['439', '440', '441', '910', '442', '443', '444', '324'] # Same order as 1-9 in project.pdf
#id_list = ['439', '440']

# Load with polars and remove noise stations 7 8 9
noise_loc = pl.read_csv('../noise_meta.csv')\
            .slice(0,6)
wea_loc = pl.read_csv('../01_Metadata_v2.csv')
wea_prep = pd.read_csv('out/wea_prep.csv')

In [49]:
# Preprocess noise level with pandas
# Using output_path fixes [Errno 13]
def noise_preprocess_csv(folder_path, output_path, names):
    noise = pd.DataFrame()
    for name in names:
        noise_row = pd.DataFrame()
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.csv') and name in file_name:
                file_path = os.path.join(folder_path, file_name)
                add = pd.read_csv(file_path, delimiter=';')

                add['date'] = add['result_timestamp'].str[0:10]
                add = add.groupby(['description', 'date'])['lamax'].mean().reset_index()

                add = add.pivot(index='description', columns='date', values='lamax')

                noise_row = pd.concat([noise_row, add], axis=1, sort=True)
        noise = pd.concat([noise, noise_row], axis=0, sort=False)
        noise['description'] = mp_list
    noise.to_csv(os.path.join(output_path, 'noise-by-day2.csv'), index=False)

    return 0

In [ ]:
noise_preprocess_csv('../noise level/', 'out/', id_list)

In [2]:
import pandas as pd

# Assuming your DataFrame is named 'df' with columns 'timestamp' and address columns
file_path = '../WentingJiang/data/processed/max_noise_per_minute_per_location.csv'
df = pd.read_csv(file_path)

# List all the address columns except 'timestamp'
address_columns = df.columns[2:]
print(address_columns)

Index(['MP 01: Naamsestraat 35  Maxim', 'MP 02: Naamsestraat 57 Xior',
       'MP 03: Naamsestraat 62 Taste', 'MP 04: His & Hears',
       'MP 05: Calvariekapel KU Leuven', 'MP 06: Parkstraat 2 La Filosovia',
       'MP 07: Naamsestraat 81', 'MP 08: bis - Vrijthof'],
      dtype='object')


In [3]:
# Loop through each address column and create a separate pivoted DataFrame for each address
pivoted = pd.DataFrame()
for address in address_columns:
    row=[]
    row = df[['timestamp', address]]
    row = row.pivot(columns='timestamp', values=address)
    row['description'] = address
    pivoted = pd.concat([pivoted, row], axis=0, sort=False)
pivoted.to_csv(os.path.join('out/', 'noise-by-10min.csv'), index=False)

MemoryError: Unable to allocate 28.7 GiB for an array with shape (17281, 223200) and data type float64

In [4]:
import pandas as pd

file_path = '../WentingJiang/data/processed/df_model_hourly.csv'
df = pd.read_csv(file_path)

# Assuming your DataFrame is named 'df'
df['timestamp'] = pd.to_datetime(df[['hour', 'day_of_month', 'month']].astype(str).apply(' '.join, axis=1), format='%H %d %m').dt.strftime('%H %d/%m')

# Display the updated DataFrame
print(df)


      Unnamed: 0 timestamp                       location  noise_level event  \
0              0  00 03/01  MP 01: Naamsestraat 35  Maxim         70.0   NaN   
1             60  01 03/01  MP 01: Naamsestraat 35  Maxim         75.7   NaN   
2            120  02 03/01  MP 01: Naamsestraat 35  Maxim         68.1   NaN   
3            180  03 03/01  MP 01: Naamsestraat 35  Maxim         72.3   NaN   
4            240  17 03/01  MP 01: Naamsestraat 35  Maxim         72.2   NaN   
...          ...       ...                            ...          ...   ...   
6903      427539  19 12/12         MP 07: Naamsestraat 81         68.4   NaN   
6904      427599  20 12/12         MP 07: Naamsestraat 81         62.7   NaN   
6905      427659  21 12/12         MP 07: Naamsestraat 81         72.1   NaN   
6906      427719  22 12/12         MP 07: Naamsestraat 81         62.1   NaN   
6907      427779  23 12/12         MP 07: Naamsestraat 81         72.2   NaN   

      count  hour day_of_week  humidity

In [13]:
pivoted = df.pivot(index='location', columns='timestamp', values='noise_level')
pivoted['description'] = mp_list2
pivoted.to_csv(os.path.join('out/', 'noise-by-hour.csv'), index=False)

In [6]:
print(pivoted)

timestamp                         00 03/01  00 03/02  00 03/03  00 03/04  \
location                                                                   
MP 01: Naamsestraat 35  Maxim         70.0      70.9      70.1      70.9   
MP 02: Naamsestraat 57 Xior           50.8      64.3      63.9      60.0   
MP 03: Naamsestraat 62 Taste          52.0      61.8      66.3      71.4   
MP 05: Calvariekapel KU Leuven        66.7      84.0      67.8      68.9   
MP 06: Parkstraat 2 La Filosovia      65.5      68.1      67.7      64.0   
MP 07: Naamsestraat 81                67.6      69.3      65.8      70.1   

timestamp                         00 03/05  00 03/06  00 03/07  00 03/08  \
location                                                                   
MP 01: Naamsestraat 35  Maxim         67.1      56.1      64.4      70.9   
MP 02: Naamsestraat 57 Xior           66.7      56.6      63.0      68.0   
MP 03: Naamsestraat 62 Taste          64.4      67.6      53.9      62.7   
MP 05: Calv